In [129]:
# ! pip install transformers scikit-learn torch

In [130]:
import pymysql
import json
import requests
import jellyfish
from datetime import datetime
import re
from difflib import SequenceMatcher as SM
import pandas as pd

In [34]:
timeout = 10
connection = pymysql.connect(
  charset="utf8mb4",
  connect_timeout=timeout,
  cursorclass=pymysql.cursors.DictCursor,
  db="iia_group2",
  host="world-athletics-world-athletics.h.aivencloud.com",
  password="AVNS_q4qfMZEYhnIhreAFwYi",
  read_timeout=timeout,
  port=24936,
  user="avnadmin",
  write_timeout=timeout,
)


In [4]:
def matcher(col1,col_list):
    m_score = -1
    m_col = None
    for i in col_list:
        sim=SM(isjunk=None, a=col1, b=i).ratio()
        if sim>m_score:
            m_score = sim
            m_col = i
    if(m_score < 0.4):
        return None, m_score
    return m_col, m_score


/home/mint/hypenX/InfoGov/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
cursor = connection.cursor()
cursor.execute("desc GST_Payers_PerYear_Data_NS")
x = cursor.fetchall()
sql_fields = [i["Field"] for i in x]
cursor.close()

In [ ]:
df1 = pd.read_csv("trans_register_payers_per_year_data.csv",index_col=0)

In [7]:
df2 = pd.read_csv("trans_GST_info_per_year_data.csv",index_col=0)

In [8]:
schema_incoming_1 = list(df1.keys())
schema_incoming_2 = list(df2.keys())

In [9]:
schema_map_1 = {}
schema_map_2 = {}

for i in schema_incoming_1:
        schema_map_1[i] = matcher(i,sql_fields)[0]
        if(schema_map_1[i] == None):
                del schema_map_1[i]
           
for i in schema_incoming_2:
        schema_map_2[i] = matcher(i,sql_fields)[0]
        if(schema_map_2[i] == None):
                del schema_map_2[i]


In [10]:
schema_map_2

{'state': 'state',
 'start_date': 'start_date',
 'end_date': 'end_date',
 'Normal tax payers': 'normal_tax_payers',
 'Composition tax payers': 'composite_tax_payer',
 'Casual tax payers': 'casual_tax_payer',
 'Non resident taxpayers': 'nri_tax_payers'}

In [11]:
df1

,state,Gst ( goods and service tax ) payers registered,Payer eligible for gst ( goods and service tax ) registration,start_date,end_date
0,andaman and nicobar islands,2596.500000,3388.833333,2017-04-01,2018-03-01
1,andaman and nicobar islands,2871.958333,4107.000000,2018-04-01,2019-03-01
2,andaman and nicobar islands,2914.541667,3570.041667,2019-04-01,2020-03-01
3,andaman and nicobar islands,3049.166667,3627.708333,2020-04-01,2021-03-01
4,andaman and nicobar islands,3207.208333,3562.750000,2021-04-01,2022-03-01
...,...,...,...,...,...
211,west bengal,375588.041667,437264.333333,2018-04-01,2019-03-01
212,west bengal,411276.708333,492672.916667,2019-04-01,2020-03-01
213,west bengal,408896.416667,511896.625000,2020-04-01,2021-03-01
214,west bengal,404378.250000,455467.500000,2021-04-01,2022-03-01


In [12]:
for i in df1.keys():
    try:
        df1[schema_map_1[i]] = df1[i]
    except KeyError:
        continue

for i in df2.keys():
    try:
        df2[schema_map_2[i]] = df2[i]
    except KeyError:
        continue

In [13]:
df_1 = df1[list(schema_map_1.values())]
df_2 = df2[list(schema_map_2.values())]

In [14]:
primary_keys = ["state","start_date","end_date"]

In [77]:
schema_map_2

{'state': 'state',
 'start_date': 'start_date',
 'end_date': 'end_date',
 'Normal tax payers': 'normal_tax_payers',
 'Composition tax payers': 'composite_tax_payer',
 'Casual tax payers': 'casual_tax_payer',
 'Non resident taxpayers': 'nri_tax_payers'}

In [80]:
insert = {}
update = {}
for i in sql_fields:
    insert[i] = []
    update[i] = []
print(schema_map_1.values())
for  _, i in df_1.iterrows():
    cursor = connection.cursor()
    cursor.execute(
        "select %s from GST_Payers_PerYear_Data_NS where state = %s AND start_date = %s AND end_date = %s",
        (
            ','.join(list(schema_map_1.values())),
            i["state"],
            i["start_date"],
            i["end_date"]
            )
    )
    x = cursor.fetchall()
    if(len(x) == 0):
        for j in insert.keys():
            try:
                insert[j].append(i[j])
            except KeyError:
                insert[j].append(None)
    else:
        x = x[0]
        for j in insert.keys():
            try:
                if(x[j] == None):
                    update[j].append(i[j])
                elif(x[j] != i[j]):
                    update[j].append(i[j])
                else:
                    update[j].append(None)
            except KeyError:
                update[j].append(None)
    cursor.close()

dict_values(['state', 'total_registered_payers', 'start_date', 'end_date'])


In [81]:
update

{'id': [None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  

In [64]:
for _, i in pd.DataFrame(insert).drop("id",axis=1).iterrows():
    try:
        cursor = connection.cursor()
        cursor.execute(
            "insert into GST_Payers_PerYear_Data_NS ("+', '.join(sql_fields[1:])+") values ("+ ', '.join(["%s" for i in sql_fields[1:]]) +")",
            tuple(i.to_list())
        )
        connection.commit()
        cursor.close()
    except:
        continue

for _, i in pd.DataFrame(update).iterrows():
    try:
        cursor = connection.cursor()
        final  = []
        final_tup = []
        for j in i.keys():
            if(i[j] != None):
                final.append(' '.join([str(j),"=","%s"]))
                final_tup.append(i[j])
        final = ', '.join(final)
        final_tup.extend([i["state"],i["start_date"],i["end_date"]])
        # print(final_tup)
        cursor.execute(
            f"update GST_Payers_PerYear_Data_NS set {final} where state = %s AND start_date = %s AND end_date = %s",
            tuple(final_tup)
        )
        connection.commit()
        cursor.close()
    except:
        continue


('andaman and nicobar islands', '2017-04-01', '2018-03-01', None, 2596.5, None, None, None, None)
('andaman and nicobar islands', '2018-04-01', '2019-03-01', None, 2871.958333333333, None, None, None, None)
('andaman and nicobar islands', '2019-04-01', '2020-03-01', None, 2914.5416666666665, None, None, None, None)
('andaman and nicobar islands', '2020-04-01', '2021-03-01', None, 3049.166666666667, None, None, None, None)
('andaman and nicobar islands', '2021-04-01', '2022-03-01', None, 3207.208333333333, None, None, None, None)
('andaman and nicobar islands', '2022-04-01', '2023-03-01', None, 2669.25, None, None, None, None)
('andhra pradesh', '2017-04-01', '2018-03-01', None, 175600.72222222222, None, None, None, None)
('andhra pradesh', '2018-04-01', '2019-03-01', None, 198456.75, None, None, None, None)
('andhra pradesh', '2019-04-01', '2020-03-01', None, 218305.75, None, None, None, None)
('andhra pradesh', '2020-04-01', '2021-03-01', None, 223499.5833333333, None, None, None, Non

In [102]:
insert = {}
update = {}
for i in sql_fields[1:]:
    insert[i] = []
    update[i] = []
print(schema_map_2.values())
for  _, i in df_2.iterrows():
    cursor = connection.cursor()
    cursor.execute(
        "select * from GST_Payers_PerYear_Data_NS where state = %s AND start_date = %s AND end_date = %s",
        (
            i["state"],
            i["start_date"],
            i["end_date"]
            )
    )
    x = cursor.fetchall()
    if(len(x) == 0):
        for j in insert.keys():
            try:
                insert[j].append(i[j])
            except KeyError:
                insert[j].append(None)
    else:
        x = x[0]
        for j in update.keys():
            try:
                if(x[j] == None and j not in primary_keys):
                    update[j].append(i[j])
                elif(x[j] != i[j] or j in primary_keys):
                    update[j].append(i[j])
                else:
                    update[j].append(None)
            except KeyError:
                update[j].append(None)
    cursor.close()

dict_values(['state', 'start_date', 'end_date', 'normal_tax_payers', 'composite_tax_payer', 'casual_tax_payer', 'nri_tax_payers'])


In [ ]:
for _, i in pd.DataFrame(insert).iterrows():
    try:
        cursor = connection.cursor()
        cursor.execute(
            "insert into GST_Payers_PerYear_Data_NS ("+', '.join(sql_fields[1:])+") values ("+ ', '.join(["%s" for i in sql_fields[1:]]) +")",
            tuple(i.to_list())
        )
        connection.commit()
        cursor.close()
    except:
        continue

for _, i in pd.DataFrame(update).iterrows():
    try:
        cursor = connection.cursor()
        final  = []
        final_tup = []
        for j in i.keys():
            if(i[j] != None):
                final.append(' '.join([str(j),"=","%s"]))
                final_tup.append(i[j])
        final = ', '.join(final)
        final_tup.extend([i["state"],i["start_date"],i["end_date"]])
        cursor.execute(
            f"update GST_Payers_PerYear_Data_NS set {final} where state = %s AND start_date = %s AND end_date = %s",
            tuple(final_tup)
        )
        connection.commit()
        cursor.close()
    except:
        continue


In [104]:
datetime.strptime(i["start_date"], "%Y-%m-%d").date()

datetime.date(2017, 4, 1)